# Parallel Programming Project

## Import Libraries

In [1]:
from PIL import Image, ImageEnhance
import os, os.path
import time
import sys
import threading

In [2]:
IMG_CNT = 0

## Functions for Bulk Image Enhancement Process

### Get inputs

In [3]:
# This function gets input from the user
def getInput():
    input_folder = input("Enter input folder path: ")
    output_folder = input("Enter output folder path: ")
    time_limit = float(input("Enter enhancing time limit: "))
    brightness = float(input("Enter brightness factor: "))
    sharpness = float(input("Enter sharpness factor: "))
    contrast = float(input("Enter contrast factor: "))
    num_threads = int(input("Enter number of threads to use: "))

    return input_folder, output_folder, time_limit, brightness, sharpness, contrast, num_threads

### Get images from input folder

In [4]:
# This function is responsible for getting the set of images from the input folder.
def getImages(path):
    imageList = []
    valid_images = [".jpeg", ".jpg", ".gif",".png"]
    
    # Iterates through all images/files in the input folder
    for i in os.listdir(path):
        ext = os.path.splitext(i)[1]
        #checks if the image is in jpeg, jpg, gif, or png format
        if ext.lower() not in valid_images:
            continue
        imageList.append((str(i),Image.open(os.path.join(path,i))))

    return imageList

### Image Enhancement Process

In [5]:
# This function is responsible for enhancing the images
def enhance(img, output_folder, brightness, sharpness, contrast, lock, startTime, time_limit, shared_counter):

    global IMG_CNT

    isEnd = False

    while img and isEnd == False:

        # Checks if time limit has been reached
        current_time = time.time()
        if current_time - startTime > time_limit:
            print("Time limit has been reached!")
            isEnd = True
            sys.exit()

        # Pop the current image to remove it from the list of images to avoid redundant image enhancement
        lock.acquire()
        currImg = img.pop()
        lock.release()

        # Get file name and file extension of the image being enhanced
        filename, file_extension = os.path.splitext(currImg[0])
        print("Enhancing " + filename)

        # Initialize ImageEnhancers for each property
        b = ImageEnhance.Brightness(currImg[1])
        s = ImageEnhance.Sharpness(currImg[1])
        c = ImageEnhance.Contrast(currImg[1])

        # Enhance image
        new_img = b.enhance(brightness)
        new_img = s.enhance(sharpness)
        new_img = c.enhance(contrast)

        # Save image
        new_img.save(output_folder + '/' + os.path.basename(filename) + file_extension)

        # Increment image enhanced counter; lock was placed
        lock.acquire()
        IMG_CNT += 1
        print("Saved " + filename)
        lock.release()

### Save to text file

In [6]:
# This function is responsible for saving the results in a text file.
def saveToTxt(outputFolder, startTime, timeLimit, shared_counter):
    
    global IMG_CNT
    # Calculates the total processing time of all images
    totalTime = time.time() - startTime
    
    # Create text file containing the results
    f = open('result.txt', 'w')
    f.write("folder location: " + outputFolder +"\nnumber of images enhanced = " + str(IMG_CNT))
    f.close()
    
    print("Total Processing Time: " + str(totalTime))
    print("Program finished executing.")

### Creation of Threads

In [7]:
# This function is responsible in creating the processes for parallel programming.
def createThreads(imageList, outputFolder, timeLimit, brightness, sharpness, contrast, nThreads):

    # List of threads object
    threads = []
    lock = threading.Lock()

    startTime = time.time()
    shared_counter = 0

    # List of threads implementation
    for i in range(0, nThreads):
        t = threading.Thread(target=enhance, args=(imageList, outputFolder, brightness, sharpness, contrast, lock, startTime, timeLimit, shared_counter))
        threads.append(t)

    # Start the execution of all threads
    for t in threads:
        t.start();

    for t in threads:
        t.join()

    # Save results to text file
    saveToTxt(outputFolder, startTime, timeLimit, shared_counter)
    

## Main Function

In [8]:
if __name__ == '__main__':
    input_folder, output_folder, time_limit, brightness, sharpness, contrast, num_threads = getInput()
    createThreads(getImages(input_folder), output_folder, time_limit, brightness, sharpness, contrast, num_threads)
    #createThreads(getImages('input'), 'output', 10, 0.2, 0.2, 0.2, 1)

Enhancing kirmizi 988
Enhancing kirmizi 1015
Enhancing kirmizi 1012
Enhancing kirmizi 981
Enhancing kirmizi 986
Saved kirmizi 988
Enhancing kirmizi 1052
Saved kirmizi 986
Enhancing kirmizi 965
Saved kirmizi 1015
Enhancing kirmizi 1029
Saved kirmizi 981
Enhancing kirmizi 1055
Saved kirmizi 1012
Enhancing kirmizi 1027
Saved kirmizi 1055
Enhancing kirmizi 1020
Saved kirmizi 965
Enhancing kirmizi 1031
Saved kirmizi 1029
Enhancing kirmizi 1036
Saved kirmizi 1052
Enhancing kirmizi 974
Saved kirmizi 1027
Enhancing kirmizi 1038
Saved kirmizi 1036
Enhancing kirmizi 1044
Saved kirmizi 1031
Enhancing kirmizi 1043
Saved kirmizi 1020
Enhancing kirmizi 973
Saved kirmizi 1038
Enhancing kirmizi 1167
Saved kirmizi 974
Enhancing kirmizi 997
Saved kirmizi 973
Enhancing kirmizi 990
Saved kirmizi 1044
Enhancing kirmizi 1003
Saved kirmizi 1167
Enhancing kirmizi 1004
Saved kirmizi 1043
Enhancing kirmizi 999
Saved kirmizi 997
Enhancing kirmizi 1021
Saved kirmizi 1004
Enhancing kirmizi 1026
Saved kirmizi 999
E

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3c0fff9d-8104-4c49-aab8-5f514b9c0ef4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>